# Project : Investigate the TMDB movie dataset : 

> Report version 1.1 : Dec 08, 2017

## Table of Contents
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#wrangling">Data Wrangling</a></li>
<li><a href="#eda">Exploratory Data Analysis</a></li>
<li><a href="#conclusions">Conclusions</a></li>
</ul>

<a id='intro'></a>
## Introduction

The Movie Database (TMDb) is a crowd-source collaborative database about movies. The dataset which we will analyze in this analysis contains about 10K movies.The dataset includes the following information:

- ID, IMDB_ID : used to identify each movie
- Movie details like : 
    - Release date and Release Year
    - Adjusted budget and revenue, which we will use to calculate profit for the movie
    - Popularity score, vote count, vote average to determine movie popularity
    - Title, cast, production companies, director, runtime, etc. 
- Certain columns in the dataset like genres, production companies have multiple values separated by "|"

### Research questions

>are **more** movies being reviewed in the TMDB dataset by release year? In which year were the highest number of movies  reviewed?

>**average** TMDb popularity by release year?

>movie with the **highest** popularity score? Display the top 20 movies by popularity score

>is budget an **indication** of the revenue made by the movie?

>If profit is calculated as the difference between revenue and budget, does mean profit have a trend over time?

>Which movie has the highest profit? Display the top 100 movies by profit made

<a id='wrangling'></a>
## Data Wrangling
Before loading the dataset, we will load all the required packages

In [96]:
import pandas as pd
import numpy as np
import scipy.special

# for plotting
from bokeh.plotting import figure,ColumnDataSource
from bokeh.io import output_notebook, show
from bokeh.models import HoverTool
output_notebook()

# select a palette
from bokeh.palettes import Dark2_5 as palette

# itertools handles the cycling
import itertools

Loading BokehJS ...

Loading the dataset from the csv and calling the data "movies"

In [97]:
movies = pd.read_csv('tmdb-movies.csv')
movies.head()

,id,imdb_id,popularity,budget,revenue,original_title,cast,homepage,director,tagline,...,overview,runtime,genres,production_companies,release_date,vote_count,vote_average,release_year,budget_adj,revenue_adj
0,135397,tt0369610,32.985763,150000000,1513528810,Jurassic World,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,http://www.jurassicworld.com/,Colin Trevorrow,The park is open.,...,Twenty-two years after the events of Jurassic ...,124,Action|Adventure|Science Fiction|Thriller,Universal Studios|Amblin Entertainment|Legenda...,6/9/15,5562,6.5,2015,1.379999e+08,1.392446e+09
1,76341,tt1392190,28.419936,150000000,378436354,Mad Max: Fury Road,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...,http://www.madmaxmovie.com/,George Miller,What a Lovely Day.,...,An apocalyptic story set in the furthest reach...,120,Action|Adventure|Science Fiction|Thriller,Village Roadshow Pictures|Kennedy Miller Produ...,5/13/15,6185,7.1,2015,1.379999e+08,3.481613e+08
2,262500,tt2908446,13.112507,110000000,295238201,Insurgent,Shailene Woodley|Theo James|Kate Winslet|Ansel...,http://www.thedivergentseries.movie/#insurgent,Robert Schwentke,One Choice Can Destroy You,...,Beatrice Prior must confront her inner demons ...,119,Adventure|Science Fiction|Thriller,Summit Entertainment|Mandeville Films|Red Wago...,3/18/15,2480,6.3,2015,1.012000e+08,2.716190e+08
3,140607,tt2488496,11.173104,200000000,2068178225,Star Wars: The Force Awakens,Harrison Ford|Mark Hamill|Carrie Fisher|Adam D...,http://www.starwars.com/films/star-wars-episod...,J.J. Abrams,Every generation has a story.,...,Thirty years after defeating the Galactic Empi...,136,Action|Adventure|Science Fiction|Fantasy,Lucasfilm|Truenorth Productions|Bad Robot,12/15/15,5292,7.5,2015,1.839999e+08,1.902723e+09
4,168259,tt2820852,9.335014,190000000,1506249360,Furious 7,Vin Diesel|Paul Walker|Jason Statham|Michelle ...,http://www.furious7.com/,James Wan,Vengeance Hits Home,...,Deckard Shaw seeks revenge against Dominic Tor...,137,Action|Crime|Thriller,Universal Pictures|Original Film|Media Rights ...,4/1/15,2947,7.3,2015,1.747999e+08,1.385749e+09


Let's look at the missing values across the various columns

In [98]:
print("Number of Missing Values in the Columns")
movies.isnull().sum()

Number of Missing Values in the Columns


id                         0
imdb_id                   10
popularity                 0
budget                     0
revenue                    0
original_title             0
cast                      76
homepage                7930
director                  44
tagline                 2824
keywords                1493
overview                   4
runtime                    0
genres                    23
production_companies    1030
release_date               0
vote_count                 0
vote_average               0
release_year               0
budget_adj                 0
revenue_adj                0
dtype: int64

Most of the columns which we are considering in this analysis have no missing value, so we don't need to do any data cleaning for missing values. 

Now, let's look at some statistics across the columns. 

In [99]:
movies.describe(include = 'all')

,id,imdb_id,popularity,budget,revenue,original_title,cast,homepage,director,tagline,...,overview,runtime,genres,production_companies,release_date,vote_count,vote_average,release_year,budget_adj,revenue_adj
count,10866.000000,10856,10866.000000,1.086600e+04,1.086600e+04,10866,10790,2936,10822,8042,...,10862,10866.000000,10843,9836,10866,10866.000000,10866.000000,10866.000000,1.086600e+04,1.086600e+04
unique,NaN,10855,NaN,NaN,NaN,10571,10719,2896,5067,7997,...,10847,NaN,2039,7445,5909,NaN,NaN,NaN,NaN,NaN
top,NaN,tt0411951,NaN,NaN,NaN,Hamlet,Louis C.K.,http://www.missionimpossible.com/,Woody Allen,Based on a true story.,...,No overview found.,NaN,Drama,Paramount Pictures,1/1/09,NaN,NaN,NaN,NaN,NaN
freq,NaN,2,NaN,NaN,NaN,4,6,4,45,5,...,13,NaN,712,156,28,NaN,NaN,NaN,NaN,NaN
mean,66064.177434,NaN,0.646441,1.462570e+07,3.982332e+07,NaN,NaN,NaN,NaN,NaN,...,NaN,102.070863,NaN,NaN,NaN,217.389748,5.974922,2001.322658,1.755104e+07,5.136436e+07
std,92130.136561,NaN,1.000185,3.091321e+07,1.170035e+08,NaN,NaN,NaN,NaN,NaN,...,NaN,31.381405,NaN,NaN,NaN,575.619058,0.935142,12.812941,3.430616e+07,1.446325e+08
min,5.000000,NaN,0.000065,0.000000e+00,0.000000e+00,NaN,NaN,NaN,NaN,NaN,...,NaN,0.000000,NaN,NaN,NaN,10.000000,1.500000,1960.000000,0.000000e+00,0.000000e+00
25%,10596.250000,NaN,0.207583,0.000000e+00,0.000000e+00,NaN,NaN,NaN,NaN,NaN,...,NaN,90.000000,NaN,NaN,NaN,17.000000,5.400000,1995.000000,0.000000e+00,0.000000e+00
50%,20669.000000,NaN,0.383856,0.000000e+00,0.000000e+00,NaN,NaN,NaN,NaN,NaN,...,NaN,99.000000,NaN,NaN,NaN,38.000000,6.000000,2006.000000,0.000000e+00,0.000000e+00
75%,75610.000000,NaN,0.713817,1.500000e+07,2.400000e+07,NaN,NaN,NaN,NaN,NaN,...,NaN,111.000000,NaN,NaN,NaN,145.750000,6.600000,2011.000000,2.085325e+07,3.369710e+07


Observations : 
- There is 1 imdb_id which is repeated in the dataset, imdb_id = tt0411951 should be checked and only one correct row should be retained in the dataset
- Convert the revenue and budget column to be in millions
- Calculate profit for each movie
- Can drop extra budget and revenue columns
- Popularity has a huge range of values, max is 32.98 whereas 75% of the data is still below 1.5
- ~50% of the values in the Budget and Revenue column are 0, we should remove these while analyzing the dataset
- Production Company "Paramount Pictures" has the most number of movies reviewed, 156


Before doing any data wrangling, let's check the shape of the original data.

In [100]:
movies.shape

(10866, 21)

There are 10866 rows in the dataset and 24 columns

#### Checking the IMDB_ID with 2 entries in the dataset

In [101]:
movies[movies['imdb_id'] == 'tt0411951']

,id,imdb_id,popularity,budget,revenue,original_title,cast,homepage,director,tagline,...,overview,runtime,genres,production_companies,release_date,vote_count,vote_average,release_year,budget_adj,revenue_adj
2089,42194,tt0411951,0.59643,30000000,967000,TEKKEN,Jon Foo|Kelly Overton|Cary-Hiroyuki Tagawa|Ian...,NaN,Dwight H. Little,Survival is no game,...,"In the year of 2039, after World Wars destroy ...",92,Crime|Drama|Action|Thriller|Science Fiction,Namco|Light Song Films,3/20/10,110,5.0,2010,30000000.0,967000.0
2090,42194,tt0411951,0.59643,30000000,967000,TEKKEN,Jon Foo|Kelly Overton|Cary-Hiroyuki Tagawa|Ian...,NaN,Dwight H. Little,Survival is no game,...,"In the year of 2039, after World Wars destroy ...",92,Crime|Drama|Action|Thriller|Science Fiction,Namco|Light Song Films,3/20/10,110,5.0,2010,30000000.0,967000.0


By looking at all the values, we see that all the values are duplicate, so we can drop either row. Let's drop index 2090 from the dataset and renaming the dataset movies_1 and the row count of this dataset should be one less than before. 

In [102]:
movies[movies['imdb_id'] == 'tt0411951'].index

Int64Index([2089, 2090], dtype='int64')

In [103]:
movies_1 = movies.drop([2090])

In [104]:
movies_1.shape

(10865, 21)

#### Changing (dollar) columns to be in millions and adding a profit column to the dataset

Since we will be doing the budget and revenue computations on the adjusted columns, let's divide these columns by 1 million, so that we get the number in millions.

In [10]:
movies_1['Revenue in millions'] = (movies_1['revenue_adj'].astype(float)/1000000)
movies_1['Budget in millions'] = (movies_1['budget_adj'].astype(float)/1000000)

Create a column called "Profit in millions" which is the difference between revenue and budget

In [11]:
movies_1['Profit in millions'] = movies_1['Revenue in millions'] - movies_1['Budget in millions']

#### Dropping unwanted dollar columns

Since we have calculated column in millions based on the adjusted numbers, we can remove the other budget and revenue columns from the dataset and name it movies_2. We added 3 new columns and dropping 4, so the column count should drop to 20. 

In [12]:
movies_2 = movies_1.drop(['revenue','budget','revenue_adj','budget_adj'],axis = 1)

In [13]:
movies_1.shape

(10865, 24)

In [14]:
movies_2.shape

(10865, 20)

<a id='eda'></a>
## Exploratory Data Analysis

Now, that we have first cut of the data, let's look at the first question. 

**_Research Question 1_** : are more movies being reviewed in the TMDB dataset by release year? In which year were the highest number of movies reviewed?

Let's start by getting a count of movies by release year and we can plot a time series to check the trend of movies reviewed. 

In [92]:
number_of_releases_by_year = movies_2.groupby('release_year')['imdb_id'].count()
plot = figure(title='Count of Movies vs. time',x_axis_label='Release Year', y_axis_label='Count of Movies')

plot.line(
    x=number_of_releases_by_year.index.values,
    y=number_of_releases_by_year.values
)
plot.circle(
    x=number_of_releases_by_year.index.values,
    y=number_of_releases_by_year.values
)

show(plot)

From the trend, there is an exponential increase in the number of movies, with a drop seen in the latest data point 2016. The drop seen in 2016 could be temporary too, similar to what we had seen in 2010. 

In [16]:
number_of_releases_by_year.sort_values(ascending = False).nlargest(10)

release_year
2014    699
2013    658
2015    628
2012    586
2011    540
2009    531
2008    496
2010    488
2007    436
2006    408
Name: imdb_id, dtype: int64

In [17]:
number_of_releases_by_year.sort_values(ascending = True)[:10]

release_year
1961    31
1969    31
1960    32
1962    32
1963    34
1965    35
1968    39
1967    40
1972    40
1970    41
Name: imdb_id, dtype: int64

By looking at the highest and lowest tables above, we see that the number of movies has increased from 30s in 1960 to 600s in 2014. 

In the data used in the analysis, 699 movies were reviewed in 2014, which is the highest in the data.  

**_Research Question 2_** : average TMDb popularity by release year

Let's calculate the mean by release years and plot the trend across the years! Before that let's create a function to plot time series. 

In [18]:
# Function to plot line chart with markers

def plot_timeseries(data):
    plot.line(
        x=data.index.values,
        y=data.values
    )
    plot.circle(
        x=data.index.values,
        y=data.values
    )
    show(plot)

In [19]:
mean_of_popularity_by_year = movies_2.groupby('release_year')['popularity'].mean()
plot = figure(title='Mean of Popularity vs. Time',x_axis_label='Release Year', y_axis_label='Mean of Popularity')
plot_timeseries(mean_of_popularity_by_year)

The mean of popularity score is 0.64, and we see the mean has been on the higher side for most years since 2000.In the last 5 years, we have seen the mean shoot up a lot. 

**_Research Question 3_** : movie with the highest popularity score? Display the top 20 movies by popularity score

Let's look at the top 20 highest popular movies. 

Jurassic World (2015) has a popularity score of 32.98.

In [20]:
mostpopularmovies = movies_2.sort_values('popularity', ascending = False).head(20)

factors = mostpopularmovies['original_title'].values
x = mostpopularmovies['popularity'].values

dot = figure(title="Top 20 popular movies", tools="", toolbar_location=None,
            y_range=factors, x_range=[0,50])

dot.segment(0, factors, x, factors, line_width=2, line_color="green", )
dot.circle(x, factors, size=15, fill_color="red", line_color="green", line_width=3, )

show(dot)

Let's spend some time to investigate the column popularity. 0.64 is the mean of the data, while the max is around 30. 
Since in the earlier part we saw 75% of the data had values less than 1 and the maximum is 30, we will bucket the popularity score to make analysis easier. We will create a function which will create a corresponding popularity category based on the range inputted in the bins variable. 

#### Adding a new column popularity category based on user specified bins

In [21]:
# Function to convert the score to bin
def convert_popularity(popularity):
        # Pandas has a bult-in function that will perform this calculation
        # more about the qcut() function here:
        # http://pandas.pydata.org/pandas-docs/stable/generated/pandas.qcut.html
        return pd.cut(popularity,bins)

In [22]:
bins = [0,0.25,0.5,0.75,5,34]
movies_2['popularity_category'] = convert_popularity(movies_2['popularity'])

In [23]:
movies_2.head()

,id,imdb_id,popularity,original_title,cast,homepage,director,tagline,keywords,overview,...,genres,production_companies,release_date,vote_count,vote_average,release_year,Revenue in millions,Budget in millions,Profit in millions,popularity_category
0,135397,tt0369610,32.985763,Jurassic World,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,http://www.jurassicworld.com/,Colin Trevorrow,The park is open.,monster|dna|tyrannosaurus rex|velociraptor|island,Twenty-two years after the events of Jurassic ...,...,Action|Adventure|Science Fiction|Thriller,Universal Studios|Amblin Entertainment|Legenda...,6/9/15,5562,6.5,2015,1392.445893,137.999939,1254.445953,"(5.0, 34.0]"
1,76341,tt1392190,28.419936,Mad Max: Fury Road,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...,http://www.madmaxmovie.com/,George Miller,What a Lovely Day.,future|chase|post-apocalyptic|dystopia|australia,An apocalyptic story set in the furthest reach...,...,Action|Adventure|Science Fiction|Thriller,Village Roadshow Pictures|Kennedy Miller Produ...,5/13/15,6185,7.1,2015,348.161292,137.999939,210.161353,"(5.0, 34.0]"
2,262500,tt2908446,13.112507,Insurgent,Shailene Woodley|Theo James|Kate Winslet|Ansel...,http://www.thedivergentseries.movie/#insurgent,Robert Schwentke,One Choice Can Destroy You,based on novel|revolution|dystopia|sequel|dyst...,Beatrice Prior must confront her inner demons ...,...,Adventure|Science Fiction|Thriller,Summit Entertainment|Mandeville Films|Red Wago...,3/18/15,2480,6.3,2015,271.619025,101.199955,170.419070,"(5.0, 34.0]"
3,140607,tt2488496,11.173104,Star Wars: The Force Awakens,Harrison Ford|Mark Hamill|Carrie Fisher|Adam D...,http://www.starwars.com/films/star-wars-episod...,J.J. Abrams,Every generation has a story.,android|spaceship|jedi|space opera|3d,Thirty years after defeating the Galactic Empi...,...,Action|Adventure|Science Fiction|Fantasy,Lucasfilm|Truenorth Productions|Bad Robot,12/15/15,5292,7.5,2015,1902.723130,183.999919,1718.723211,"(5.0, 34.0]"
4,168259,tt2820852,9.335014,Furious 7,Vin Diesel|Paul Walker|Jason Statham|Michelle ...,http://www.furious7.com/,James Wan,Vengeance Hits Home,car race|speed|revenge|suspense|car,Deckard Shaw seeks revenge against Dominic Tor...,...,Action|Crime|Thriller,Universal Pictures|Original Film|Media Rights ...,4/1/15,2947,7.3,2015,1385.748801,174.799923,1210.948878,"(5.0, 34.0]"


In the first question, we had seen that the number of movies had exponentially increased over time. 

In [24]:
number_of_movies_by_popcat = movies_2.groupby('popularity_category')['popularity'].count()
uniquepopcat = number_of_movies_by_popcat.index.values

In [25]:
# create a color iterator
colors = itertools.cycle(palette) 

hover = HoverTool(tooltips=[
    ("Count of movies", "$y{int}")])

plot = figure(title='Count of Movies vs. time',x_axis_label='Release Year', y_axis_label='Count of Movies',tools=[hover])

for cat, color in zip(uniquepopcat, colors):
    data = movies_2[movies_2['popularity_category'] == cat].groupby('release_year')['imdb_id'].count()
    _x=data.index.values
    _y=data.values
    data = {'x':_x, 'y':_y}
    source = ColumnDataSource(data)

    plot.line(_x, _y, legend='category = {}'.format(cat), color=color)
    plot.circle(_x, _y, color=color)
    
plot.legend.location='top_left'    
show(plot)

From the chart above, we see that increase in movie count is seen across popularity categories. Prior to 2000, most of the movies had popularity score between 0.25 to 0.5. But since 2010, we are seeing a huge increase in movies with very low popularity score, less than 0.25.

Also, in the recent years we are seeing an increase with movies with very high popularity score. As we had seen earlier, the movies with the highest score do look to be the popular movies. 

**_Research Question 4_** : is budget an indication of the revenue made by the movie?

We saw that around 50% of the budget for the movies are reported as zero. Instead of removing this data, let's replace the 0s with the average budget from the data. 

In [26]:
movies_2.describe()

,id,popularity,runtime,vote_count,vote_average,release_year,Revenue in millions,Budget in millions,Profit in millions
count,10865.000000,10865.000000,10865.000000,10865.000000,10865.000000,10865.000000,10865.000000,10865.000000,10865.000000
mean,66066.374413,0.646446,102.071790,217.399632,5.975012,2001.321859,51.369002,17.549894,33.819108
std,92134.091971,1.000231,31.382701,575.644627,0.935138,12.813260,144.638333,34.307527,125.219384
min,5.000000,0.000065,0.000000,10.000000,1.500000,1960.000000,0.000000,0.000000,-413.912431
25%,10596.000000,0.207575,90.000000,17.000000,5.400000,1995.000000,0.000000,0.000000,0.000000
50%,20662.000000,0.383831,99.000000,38.000000,6.000000,2006.000000,0.000000,0.000000,0.000000
75%,75612.000000,0.713857,111.000000,146.000000,6.600000,2011.000000,33.701729,20.853251,12.930908
max,417859.000000,32.985763,900.000000,9767.000000,9.200000,2015.000000,2827.123750,425.000000,2750.136651


#### Replace 0 budgets with mean value

In [111]:
mean_budget = movies_2['Budget in millions'].mean()
movies_3 = movies_2.replace({'Budget in millions': {0: mean_budget}}) 


In [112]:
movies_3.describe()

,id,popularity,runtime,vote_count,vote_average,release_year,Revenue in millions,Budget in millions,Profit in millions
count,10865.000000,10865.000000,10865.000000,10865.000000,10865.000000,10865.000000,10865.000000,1.086500e+04,10865.000000
mean,66066.374413,0.646446,102.071790,217.399632,5.975012,2001.321859,51.369002,2.675046e+01,33.819108
std,92134.091971,1.000231,31.382701,575.644627,0.935138,12.813260,144.638333,3.051007e+01,125.219384
min,5.000000,0.000065,0.000000,10.000000,1.500000,1960.000000,0.000000,9.210911e-07,-413.912431
25%,10596.000000,0.207575,90.000000,17.000000,5.400000,1995.000000,0.000000,1.754989e+01,0.000000
50%,20662.000000,0.383831,99.000000,38.000000,6.000000,2006.000000,0.000000,1.754989e+01,0.000000
75%,75612.000000,0.713857,111.000000,146.000000,6.600000,2011.000000,33.701729,2.085325e+01,12.930908
max,417859.000000,32.985763,900.000000,9767.000000,9.200000,2015.000000,2827.123750,4.250000e+02,2750.136651


In [113]:
mean_of_budget_by_year = movies_2.groupby('release_year')['Budget in millions'].mean()
mean_of_budget_by_year_nz = movies_3.groupby('release_year')['Budget in millions'].mean()

plot = figure(title='Mean of Budget vs. time',x_axis_label='Release Year', y_axis_label='Mean of Budget')

plot.line(
    x=mean_of_budget_by_year.index.values,
    y=mean_of_budget_by_year.values, 
    color = 'red'
)
plot.circle(
    x=mean_of_budget_by_year.index.values,
    y=mean_of_budget_by_year.values,
    color = 'red', 
    legend = "Non Adjusted Budget"
)

plot.line(
    x=mean_of_budget_by_year_nz.index.values,
    y=mean_of_budget_by_year_nz.values, 
    color = 'blue'
)
plot.circle(
    x=mean_of_budget_by_year_nz.index.values,
    y=mean_of_budget_by_year_nz.values,
    color = 'blue', 
    legend = "Adj budget : zero replaced with mean budget"
)
plot.legend.location='top_left'
show(plot)

We still see 50% of revenue data being 0, but a movie could theoretically make 0 revenue. So, we will keep these rows. 

For the rest of the analysis, we will use the dataset where the 0 budget values have been replaced with the mean.

Now, let's plot a scatter plot of budget vs revenue do see whether the budget of the movie determines the revenue it makes. 

In [114]:
# Plotting Revenue against Budget
budget_by_movie = movies_3.groupby('imdb_id')['Budget in millions'].mean()
revenue_by_movie = movies_3.groupby('imdb_id')['Revenue in millions'].mean()
profit_by_movie = movies_3.groupby('imdb_id')['Profit in millions'].mean()

plot = figure(title='Scatter of Budget against Revenue',x_axis_label='Budget', y_axis_label='Revenue')
plot.circle(
    x=budget_by_movie.values,
    y=revenue_by_movie.values,
    #size='size',
    #source=plot_df,
    line_color='blue',
    line_width=0.5
    #legend='labels'
)
show(plot)

From this trend, we see a very weak positive correlation between Revenue and Budget. Let's calculate the correlation of coefficient for Budget vs Revenue.

In [115]:
movies_3['Budget in millions'].corr(movies_3['Revenue in millions'])

0.63093656546944088

Revenue shows a weak positive correlation with budget. Does the correlation look different when we bin the movies by popularity category?

In [119]:
# create a color iterator
colors = itertools.cycle(palette) 

hover = HoverTool(tooltips=[
    ("Revenue in millions", "$y{int}")])

plot = figure(title='Scatter plot of Budget vs. Revenue',x_axis_label='Budget', y_axis_label='Revenue',tools=[hover])

for cat, color in zip(uniquepopcat, colors):
    data_b = movies_3[movies_3['popularity_category'] == cat].groupby('imdb_id')['Budget in millions'].mean()
    data_r = movies_3[movies_3['popularity_category'] == cat].groupby('imdb_id')['Revenue in millions'].mean()
    _x=data_b.values
    _y=data_r.values
    data = {'x':_x, 'y':_y}
    source = ColumnDataSource(data)

    plot.circle(_x, _y, legend='category = {}'.format(cat), color=color, fill_alpha = 0)
    
plot.legend.location='top_left'    
show(plot)

In [120]:
corr_popcat = {}
for cat in zip(uniquepopcat) : 
    data = movies_2[movies_2['popularity_category'] == cat]
    corr_popcat[cat] = data['Budget in millions'].corr(data['Revenue in millions'])
    
corr_popcat

{(Interval(0.0, 0.25, closed='right'),): 0.39958867194017517,
 (Interval(0.25, 0.5, closed='right'),): 0.3810719713011208,
 (Interval(0.5, 0.75, closed='right'),): 0.41957014874624166,
 (Interval(0.75, 5.0, closed='right'),): 0.58004276115005848,
 (Interval(5.0, 34.0, closed='right'),): 0.49205836569105815}

The observation remains the same. Irrespective of which popularity category we choose, the correlation is very weak between revenue and budget.

**_Research Question 5_** : If profit is calculated as the difference revenue and budget, does mean profit have a trend over time?

As part of the Data Cleaning step, we had already calculated the profit column. Let's start by looking at the average profit for the year over time.

In [121]:
mean_of_profit_by_year = movies_2.groupby('release_year')['Profit in millions'].mean()

plot = figure(title='Mean of Profit vs. time',x_axis_label='Release Year', y_axis_label='Mean of Profit')

plot.line(
    x=mean_of_profit_by_year.index.values,
    y=mean_of_profit_by_year.values, 
    color = 'red'
)
plot.circle(
    x=mean_of_profit_by_year.index.values,
    y=mean_of_profit_by_year.values,
    color = 'red', 
    legend = "all movies"
)


plot.legend.location='top_right'
show(plot)

The average profit made over the years has been on a decreasing trend. 

**_Research Question 6_** : Which movie has the highest profit? Display the top 100 movies by profit made



In [122]:
top_100_movies_by_popularity = movies_3.groupby(['original_title','release_year','popularity','genres'])['Profit in millions','Budget in millions'].mean().sort_values('Profit in millions',ascending = False).head(100)
top_100_movies_by_popularity.head()

,,,,Profit in millions,Budget in millions
original_title,release_year,popularity,genres,,
Star Wars,1977,12.037933,Adventure|Action|Science Fiction,2750.136651,39.575591
Avatar,2009,9.432768,Action|Adventure|Fantasy|Science Fiction,2586.236848,240.886903
Titanic,1997,4.355219,Drama|Romance|Thriller,2234.713671,271.692064
The Exorcist,1973,2.010733,Drama|Horror|Thriller,2128.035625,39.289277
Jaws,1975,2.563191,Horror|Thriller|Adventure,1878.643094,28.362748


Star Wars(1977) is the movie with the highest profit made. Now, let's look at the top 100 movies by highest profit.

In [123]:
# reduce data size
data2_temp = top_100_movies_by_popularity
bubble_size = data2_temp['Profit in millions'].values
b_min = min(bubble_size)
b_max = max(bubble_size)
bubble_size_plot = [(item - b_min)*100/b_max for item in bubble_size]
x = [item[1] for item in data2_temp.index.values]
y = data2_temp['Budget in millions'].values
labels = [item[0] for item in data2_temp.index.values]

In [124]:
plot_df = pd.DataFrame()
plot_df["x"] = x
plot_df["y"] = y
plot_df["size"] = bubble_size_plot
plot_df["labels"] = labels
#plot_df

In [125]:
output_notebook()

plot = figure(title='Top 100 movies by Profit, Profit=Bubble size',x_axis_label='Release Year', y_axis_label='Budget in millions ($)')
plot.circle(
    x='x',
    y='y',
    size='size',
    source=plot_df,
    fill_alpha = 0.1,
    line_color='#7c7e71',
    line_width=0.5,
    line_alpha=0.5
    #legend='labels'
)
plot.add_tools(HoverTool(tooltips="@labels", show_arrow=False, point_policy='follow_mouse'))

show(plot)


Loading BokehJS ...

Between 1970 - 1980 lots of high profit movies were made - star wars, the exorcist, the jaws, etc. 

Between 1990 - 2000, only one movie Titanic made it to the top 100 list. 

<a id='conclusions'></a>
## Conclusions

The results called out here are only observations from the TMDb dataset of 10K movies. As mentioned before, the TMDb is a small sample of all the movies that are produced worldwide. Also, movies not reviewed outside of the 10K list have not been considered.Since we have not performed any statistical test to prove/disprove correlation among the various variables. We are only stating the trend that was observed in this specific dataset. 

- The number of movies reviewed in the TMDb dataset is exponentially increasing and this trend is seen across all categories irrespective of popularity
- Budget and revenue shows a weak positive correlation. 
- The average profit on movies has been on a decreasing trend
- Jurassic Park(2015) is the most popular movie, and Star Wars(1977) made the most money

## References

* https://www.stackoverflow.com
* https://bokeh.pydata.org/en/latest/